In [1]:
import pandas as pd
import numpy as np

In [2]:
def dateStringtoint(str):
    return int(str[11:13])

In [3]:
def stratSample(path):
    returnBool = False
    
    audio = pd.read_csv(path)
    audio = audio[round(audio.Duration, 0) == 60.0]
    
    good_moths = audio.groupby(['AudioMothCode']).count().AudioMothID >= 2000
    to_keep = audio.groupby(['AudioMothCode']).count()[good_moths].index
    
    bad_moths = audio.groupby(['AudioMothCode']).count().AudioMothID < 2000
    to_drop = audio.groupby(['AudioMothCode']).count()[bad_moths].index

    for i in to_drop:
        audio = audio[audio.AudioMothCode != i]
    
    audio['HourInt'] = audio.StartDateTime.apply(dateStringtoint)
    
    gb = audio.groupby(['AudioMothCode', 'HourInt'])
    groups = [gb.get_group(x) for x in gb.groups]
    
    sampled_df = pd.DataFrame(columns=audio.columns)
    
    for group in groups:
        sampled_df = sampled_df.append(group.sample())
    
    sampled_df = sampled_df.drop(columns=['HourInt'])
    
    #if data frame is expected size return it and return true, else return empty df and false
    if len(sampled_df) == len(to_keep) * 24:
        returnBool = True
        return sampled_df.to_csv(), returnBool

    else:
        return pd.DataFrame(columns=audio.columns).to_csv(), returnBool    